In [1]:
import os
os.environ['SCIPY_ARRAY_API'] = '1'

from tabpfn_extensions.post_hoc_ensembles.sklearn_interface import AutoTabPFNRegressor
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_percentage_error
from tqdm import tqdm

def apply_tabpfn(train_fold, val_fold, df_test_pred, feature_cols, col_name):
    regressor = AutoTabPFNRegressor(max_time=120, device="cuda") # 120 seconds tuning time
    regressor.fit(train_fold[feature_cols], train_fold[col_name])
    
    print(regressor.predict(df_test_pred))
    

# df_train = pd.read_csv('./dataset/train.csv')

# train_features = df_train.columns[:55]
# target_col = 'BlendProperty3'

# X_train = 

# clf = AutoTabPFNRegressor(max_time=120, device="cuda") # 120 seconds tuning time
# clf.fit(X_train, y_train)
# predictions = clf.predict(X_test)

def apply_tabpfn_cv(df_train, df_test_pred, feature_cols, col_name, seed=42, n_splits=5):
    
    # n_splits = 5
    
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    # col_name = f'BlendProperty{target_col - 55 + 1}'
    df_oof_preds = df_train[[col_name]].copy()
    
    test_preds_avg = None
    
    for fold, (train_idx, val_idx) in tqdm(enumerate(kf.split(df_train))):
        train_fold = df_train.iloc[train_idx]
        val_fold = df_train.iloc[val_idx]
        
        score, val_preds, test_preds = apply_tabpfn(train_fold, val_fold, df_test_pred, feature_cols, col_name)
        df_oof_preds.loc[df_train.index[val_idx], col_name] = val_preds
        
        if test_preds_avg is None:
            test_preds_avg = test_preds
        else:
            test_preds_avg += test_preds
    
    test_preds_avg = test_preds_avg / n_splits
    
        # results_dir = f'./TabM/CV_b{target_col - 55 + 1}'
        # os.makedirs(results_dir, exist_ok=True)
        # np.save(f'{results_dir}/test_pred_fold_{fold}.npy', test_preds)
    
    mape = mean_absolute_percentage_error(df_train[col_name], df_oof_preds[col_name])
    score = 100 - 90 * mape / 2.72
    col_num = col_name[-1]
    if col_num == '0':
        col_num = 10
    os.makedirs(f'./tabpfn/{col_name}', exist_ok=True)
    np.save(f'./tabpfn/{col_name}/oof_b{col_num}_seed_{seed}_splits_{n_splits}_score_{score}.npy', df_oof_preds[col_name].values)
    return score, test_preds_avg


df_train = pd.read_csv('./dataset/train_small.csv')
df_test = pd.read_csv('./dataset/test.csv')
df_best = pd.read_csv('./submission_cur_best+tabm_b1234_10fold(cb)_93.92695.csv')

feature_cols = df_train.columns[:55]
df_test_pred = pd.concat([df_test, df_best], axis=1)

target_col = 'BlendProperty3'
apply_tabpfn_cv(df_train, df_test_pred, feature_cols, target_col)

0it [00:00, ?it/s]/home/nuwaisir/miniconda3/envs/shell_ai_2024_python_3.12/lib/python3.12/site-packages/sklearn/preprocessing/_data.py:3457: RuntimeWarning: overflow encountered in power
  x_inv[~pos] = 1 - np.power(-(2 - lmbda) * x[~pos] + 1, 1 / (2 - lmbda))


tabpfn-v2-regressor-wyl4o83o.ckpt:   0%|          | 0.00/23.2M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/37.0 [00:00<?, ?B/s]

/home/nuwaisir/miniconda3/envs/shell_ai_2024_python_3.12/lib/python3.12/site-packages/sklearn/preprocessing/_data.py:3457: RuntimeWarning: overflow encountered in power
  x_inv[~pos] = 1 - np.power(-(2 - lmbda) * x[~pos] + 1, 1 / (2 - lmbda))
INFO:tabpfn_extensions.post_hoc_ensembles.greedy_weighted_ensemble:Order of selections: [0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 4, 0, 0, 0, 4, 0, 0, 4, 0]
INFO:tabpfn_extensions.post_hoc_ensembles.greedy_weighted_ensemble:Val loss over iterations: [0.09354690457760213, 0.08880788835363305, 0.0868210868653599, 0.0868210868653599, 0.0868210868653599, 0.0868210868653599, 0.0868210868653599, 0.0868210868653599, 0.0868210868653599, 0.0868210868653599, 0.0868210868653599, 0.0868210868653599, 0.0868210868653599, 0.0868210868653599, 0.0868210868653599, 0.0868210868653599, 0.0868210868653599, 0.0868210868653599, 0.08681941785778964, 0.08681941785778964, 0.08681941785778964, 0.08681731847698444, 0.08681731847698444, 0.08681731847698444, 0.086816214

ValueError: X has 67 features, but ColumnTransformer is expecting 55 features as input.